I ran a simulation with parametrized sources of Q1 Q2 and Q3, but it died after less than 1 day of simulation. This notebook explores this model pathology.

In [ ]:
%matplotlib inline
import xarray as xr
import holoviews as hv
hv.extension('bokeh')

from os.path import join


def resample(src, onto):
    return src.sel(time=onto.time, method='nearest')\
        .assign_coords(x=onto.x, y=onto.y, time=onto.time)


In [ ]:
TRAINING = "../data/processed/training.nc"
no_q3_path = "../models/17/test"
train_data = xr.open_dataset(TRAINING)


data_3d = xr.open_mfdataset("../data/runs/2018-10-05-q1_q2_and_q3/OUT_3D/*.nc")
data_2d = xr.open_mfdataset("../data/runs/2018-10-05-q1_q2_and_q3/OUT_2D/*.nc")


train_data_resampled = resample(train_data, data_3d)

data_no_q3_3d = resample(xr.open_mfdataset(join(no_q3_path, "OUT_3D", "*.nc")), data_3d)

In [ ]:
data_3d.W[:,:,32].plot(col='time')

In [ ]:
data_2d.W500.plot(col='time', col_wrap=4)

There are clearly some problems with the vertical velocity near the N/S boundaries. Perhaps I should turn off the neural networks for momentum in those regions. How do the results look outside of these regions?

In [ ]:
data_2d.W500.isel(y=slice(3,-3)).plot(col='time', col_wrap=5)

Indeed, the results look reasonable in the center of the domain. Did the structure of zonal winds improve there? In this cell I plot the difference

$$ \bar{u} - \bar{u}_{OBS} $$.

In [ ]:
%%opts Curve[invert_axes=True]

u_equator = data_3d.U.isel(y=32).mean('x')
u_eq_obs = train_data_resampled.U.isel(y=32).mean('x')

plotme = xr.Dataset({'NN': u_equator, 
                     'NoQ3': data_no_q3_3d.U.isel(y=32).mean('x')}).to_array(dim='run', name='U')

lay = hv.Dataset(plotme-u_eq_obs).to.curve("z").overlay("run").layout().cols(2)
lay.redim.range(U=(-2,2))

The zonal mean wind profiles seem OK, but not great.

In [ ]:
%%opts Curve[invert_axes=True]

lay.redim.range(U=(-2, 2), z=(0,4000))

When we zoom in on the lower troposphere, we see that the additional momentum source greatly reduces the difference in the zonal means compared to the NoQ3 scheme.

# Conclusions

To stabilize the simulation, I will not apply the neural network within 3 grid points of the north south boundary.